In [ ]:
from sketch.examples.prompt_machine import *

import pandas as pd

In [ ]:
# load any data you want (including using st helper functions)
state_data = pd.read_html('https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States')[1]
state_data.columns = state_data.columns.to_flat_index()
state_names = state_data[state_data.columns[0]]

In [ ]:
state_names.head()

In [ ]:
result = state_names.apply(
    GPT3Prompt(
        "get_color_for_state",
        "Colors for NBA fans in {{ state }} (as a hex, eg. (0xF1F1F1)): (0x",
        stop=")"
    )
)

In [ ]:
import altair as alt
from vega_datasets import data
alt.renderers.enable('html')

results = pd.concat([state_names.rename("state_name"), result.rename("color_hex")], axis=1)
results.index.name = 'id'
results = results.reset_index()

# # altair state to color plot
states = alt.topo_feature(data.us_10m.url, feature='states')

alt.Chart(states).mark_geoshape().encode(
    color=alt.Color(
        "state_name:N",
        scale=alt.Scale(
            domain=list(results['state_name'].values),
            range=list(("#"+results['color_hex']).values)
        )
    ),
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(results, 'id', list(results.columns))
).properties(
    width=500,
    height=300
).project('albersUsa')

![Map Attempt](map_attempt_0.png)